In [ ]:
!pip install -U spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.5
    Uninstalling spacy-3.8.5:
      Successfully uninstalled spacy-3.8.5


In [ ]:
!pip install spacy[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json

with open('training_data.json', 'r') as f2:
  data1 = json.load(f2)

print(data1)

JSONDecodeError: Extra data: line 39 column 1 (char 717)

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [ ]:
from spacy.util import filter_spans

for train in tqdm(data):
    text = train['text']
    labels = train['entities']
    doc = nlp.make_doc(text)
    ents = []
    for ent in train["entities"]:
       start = int(ent["start"])
       end = int(ent["end"])
       label = ent["label"]
       span = doc.char_span(start, end, label=label, alignment_mode="contract")
       if span is None:
         print("Skipping entity")
       else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy") # save the docbin object

100%|██████████| 6/6 [00:00<00:00, 554.03it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [ ]:
!python -m spacy init config config.cfg --lang pt --pipeline ner --optimize efficiency --force

ℹ Generated config template specific for your use case
- Language: pt
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     42.12    4.94    2.94   15.38    0.05
101     200        104.92   1744.03  100.00  100.00  100.00    1.00
301     400          0.00      0.00  100.00  100.00  100.00    1.00
501     600          0.00      0.00  100.00  100.00  100.00    1.00
701     800          0.00      0.00  100.00  100.00  100.00    1.00
901    1000          0.00      0.00  100.00  100.00  100.00    1.00
1101    1200          0.00      0.00  100.00  100.00  100.00    1.00
1301    1400          0.00      0.00  100.00  100.00  100.00    1.00
1501    1600          0.00      0.

In [ ]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner("Claim number 12345678900 was filed by Jane Smith for a water damage loss on April 21, 2023. The policy number is POL12345678. The insurer involved is State Farm.")

colors = {"CLAIM_NUMBER": "#F67DE3", "PERSON": "#7DF6D9", "POLICY_NUMBER":"#f53803","LOSS_TYPE":"#F67DE3","DATE":"#f5d020","INSURER":"#7DF6D9"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner("John Doe filed claim number 12345678900 on April 21, 2023 for car damage loss. State Farm is the insurer involved and the policy number is POL12345678.")

colors = {"CLAIM_NUMBER": "#F67DE3", "PERSON": "#7DF6D9", "POLICY_NUMBER":"#f53803","LOSS_TYPE":"#F67DE3","DATE":"#f5d020","INSURER":"#7DF6D9"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner("On April 21, 2023, John Doe filed claim number 12345678900 for car damage loss. State Farm is the insurer involved. The policy number is POL12345678.")

colors = {"CLAIM_NUMBER": "#F67DE3", "PERSON": "#7DF6D9", "POLICY_NUMBER":"#f53803","LOSS_TYPE":"#F67DE3","DATE":"#f5d020","INSURER":"#7DF6D9"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

References
https://www.newscatcherapi.com/blog/train-custom-named-entity-recognition-ner-model-with-spacy-v3